In [1]:
import sys
sys.path.append("..")

from utils import *
from trainer import *
import scanpy as sc
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import argparse
import pandas as pd
import scipy.sparse as sp
seed=2024
fix_seed(seed)

import h5py
import anndata

In [2]:
data_file = h5py.File("/mnt/sdc/tyh/spatial/SMOPCA_data/Simulation/CITEseq/SLN111D1_rep0.h5")

In [3]:
X1 = np.array(data_file['X1'])
X2 = np.array(data_file['X2'])
pos = np.array(data_file['pos'])
y = np.array(data_file['Y'])

In [4]:
adata_omics1 = anndata.AnnData(X1)
adata_omics2 = anndata.AnnData(X2)
adata_omics1.obsm['spatial'] = pos
adata_omics2.obsm['spatial'] = pos

In [5]:
adata_omics1.write_h5ad('./RNA.h5ad')
adata_omics2.write_h5ad('./ADT.h5ad')

In [6]:
adata1_path='./RNA.h5ad'
adata2_path='./ADT.h5ad'

In [7]:
adata_omics1,adata_omics2=pre_process(adata1_path,adata2_path,data_type='Simulation',
                                      spatial_neighbors=20,
                                      omics1_neighbors=10,
                                      omics2_neighbors=10,
                                      radius=0.03
                                     )

../utils.py:37: ImplicitModificationWarning: Setting element `.obsm['adj_spatial']` of view, initializing view as actual.
  adata_omics1.obsm['adj_spatial'] = spatial_adj


In [8]:
train(
    adata_omics1, adata_omics2,
    save_path='./1.h5ad',
    n_cluster=28,
    lr=0.001,
    weight_decay=0.1,
    Epoch=5000,
    device='cuda:1'
)

Training Progress, loss: 0.7584,rec1: 0.3469,rec2: 0.2547, mmd1: 0.0002,mmd2: 0.0001, sp: 0.1557, dec: 0.0008: 100%|█| 5000/5000 [13:54<00:00,  5
R[write to console]:                    __           __ 
   ____ ___  _____/ /_  _______/ /_
  / __ `__ \/ ___/ / / / / ___/ __/
 / / / / / / /__/ / /_/ (__  ) /_  
/_/ /_/ /_/\___/_/\__,_/____/\__/   version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.



fitting ...
  |======================================================================| 100%


In [9]:
data=sc.read_h5ad('./1.h5ad')

In [10]:
print(adjusted_rand_score(y, data.obs['mclust']))

0.7597251596194343
